# Day 11

In [14]:
test_data = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1
"""


In [15]:
from adventofcode import AdventOfCode
import re
import pprint
from copy import deepcopy
import math
from tqdm import tqdm
from dataclasses import dataclass

@dataclass
class Item:
    init_worry_level: int
    worry_level: int
    monkey: int
    monkeys: list

class Day11(AdventOfCode):
    worry = False
    rounds = 20
    operations = None
    worry_divider_part_2 = 1

    def set_operations(self, operations):
        self.operations = operations

    def get_input_values(self):
        lines = self.input_data.strip().split("\n")
        print(len(lines))
        monkeys={}
        for i in range(len(lines)):
            if "Monkey" in lines[i]:
                idx = len(monkeys)
                monkeys[idx] = {
                    "items": [int(x) for x in re.findall(r'(\d+)', lines[i+1])],
                    "operation": None,  # re.search(r'.*=\s+(.*)', lines[i+2]).groups()[-1],
                    "divisor": int(re.findall(r'(\d+)', lines[i+3])[-1]),
                    "target_true": int(re.findall(r'(\d+)', lines[i+4])[-1]),
                    "target_false": int(re.findall(r'(\d+)', lines[i+5])[-1]),
                    "activity": 0
                }

        return monkeys
            
    def get_initial_items(self, monkeys):
        items = []
        for monkey in monkeys:
            for item in monkeys[monkey]["items"]:
                items.append(Item(item, item, monkey, str(monkey)))
        return items

    def iterate_on_items(self, monkeys, items):
        for round in tqdm(range(self.rounds)):
            for i in monkeys.keys():
                m = monkeys[i]
                for item in filter(lambda x: x.monkey == i, items):
                    m["activity"] += 1
                    item.worry_level = self.operations[i](item.worry_level)
                    if not self.worry:
                        item.worry_level //= 3
                    else:
                        # How to set the correct worry level? take the modulo of the product of all test divisors
                        # Test data
                        # item.worry_level %= 23*13*17*19
                        # Input data
                        item.worry_level %= self.worry_divider_part_2
                        
                    if (item.worry_level % m["divisor"] == 0):
                        item.monkey = m["target_true"]
                    else:
                        item.monkey = m["target_false"]
                    item.monkeys += str(i)

    def solve(self):
        monkeys = self.get_input_values()
        items = self.get_initial_items(monkeys)
        self.iterate_on_items(monkeys, items)
        two_most_actives = sorted([monkeys[m]['activity'] for m in monkeys], reverse=True)[:2]
        return math.prod(two_most_actives)


In [16]:
test_operations = [
        lambda x: x*19,
        lambda x: x+6,
        lambda x: x*x,
        lambda x: x+3,
]
c = Day11(test_data)
c.set_operations(test_operations)
c.run()
# 10605

27


100%|██████████| 20/20 [00:00<?, ?it/s]

Running AdventOfCode : 10605


10605

Part 1

In [17]:
from adventofcode import AdventOfCodeFromFileInput

operations = [
        lambda x: x*19,
        lambda x: x+1,
        lambda x: x+6,
        lambda x: x+5,
        lambda x: x*x,
        lambda x: x+7,
        lambda x: x*7,
        lambda x: x+2,
]

challenge = AdventOfCodeFromFileInput(Day11, "input_11.txt")
challenge.set_operations(operations)
challenge.run()

55


100%|██████████| 20/20 [00:00<?, ?it/s]

Running AdventOfCode Day11: 50830


50830

Part 2

This part was really tricky. The problem is that after 100+ iterations, the "square" operation grows the items worry level exponentially and calculation takes so much time then, it's hard to go past 100+ iterations.

First I thought a useful optimization compared to what I did in part 1 was to create a static list of items, where each item tracks its current worry level and monkey. It is way easier to manipulate than my first implementation where each monkey handled a list of items, and the algorithm needed to move items from a monkey to another, performing "deepcopies" of the lists. But in the end, this optimization alone did not change the overall result: the algorithm was still stalling after 150+ operations.

So how to get the current worry level compensation factor for this part? At first I tried to draw, for each item, the list of all monkeys that the item passes through. By analogy to a directed graph, my goal was to identify loops and "reduce" the worry level by cutting the loops. But this did not give any result, I could not identify loops in the path the items were taking.

Finally I just spent time on trial and errors and eventually find out that the solution was to take the modulo of the worry level and the product of all operations divisors (eg: `23*13*17*19` for the example).

At first I thought this was just a random choice, but after getting the solution I started reading about what others did and I think I start making some sense about it (modular arithmetic is tricky). We don't care about the worry levels themselves, only the states (monkeys) the items go through. And the transitions only depend on modulo operations (which all use prime numbers by the way).

So if we find a way to retain the information in the worry level about the modulo operations, the rest is not useful - so the operation to apply is a modulo operation on the worry level itself with an operand that will maintain the information for all possible remaining modulo operations the monkeys will use.

Let's draw a simple example:
```python
In [38]: 287%7
Out[38]: 0

In [39]: 287%3
Out[39]: 2

In [40]: 287%(3*7)
Out[40]: 14

In [41]: 14%7
Out[41]: 0

In [42]: 14%3
Out[42]: 2
```
We can see that 14 (287%(3*7)) and 287 are congruent modulo 7 and 3.

From the example we can also see that as long as the modulo operators are prime, we have the lowest value and thus the maximum "reduction" of the worry level. If one of the divisors was not prime, my guess is that the product of all divisors would still work, but would be more than what is needed, because in sumary we need the integer dividers of the divisors.

The reference that helped me understand is [Jonathan Paulson's code](https://github.com/jonathanpaulson/AdventOfCode/blob/master/2022/11.py) (also check his youtube AdventOfCode videos - impressive!). 

In [18]:
class Day11Part2(Day11):
    worry = True
    rounds = 10000
    worry_divider_part_2 = 7*19*5*11*17*13*2*3

In [19]:
c = Day11Part2(test_data)
c.set_operations(test_operations)
c.run()

27


100%|██████████| 10000/10000 [00:00<00:00, 28575.06it/s]

Running AdventOfCode : 2567194800


2567194800

In [20]:

challenge = AdventOfCodeFromFileInput(Day11Part2, "input_11.txt")
challenge.set_operations(operations)
challenge.run()

55


100%|██████████| 10000/10000 [00:01<00:00, 6265.53it/s]

Running AdventOfCode Day11Part2: 14399640002


14399640002